In [4]:
import pygame
import sys
import random
import time
import os
import math

# Khởi tạo pygame
pygame.init()
pygame.mixer.init()

# Màu sắc
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (65, 105, 225)  # Royal Blue
RED = (220, 20, 60)    # Crimson
GREEN = (34, 139, 34)  # Forest Green
GRAY = (200, 200, 200)
GOLD = (255, 215, 0)   # Gold cho hiệu ứng
PURPLE = (128, 0, 128) # Purple cho nền
LIGHT_BLUE = (173, 216, 230)  # Light Blue cho hiệu ứng

# Thiết lập màn hình
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Memory Game")

# Font chữ
try:
    font = pygame.font.Font(None, 36)
    small_font = pygame.font.Font(None, 24)
    title_font = pygame.font.Font(None, 60)
except:
    font = pygame.font.SysFont(None, 36)
    small_font = pygame.font.SysFont(None, 24)
    title_font = pygame.font.SysFont(None, 60)

# Âm thanh
try:
    correct_sound = pygame.mixer.Sound("sounds\\correct-156911.mp3")
    wrong_sound = pygame.mixer.Sound("sounds\\buzzer-or-wrong-answer-20582.mp3")
    level_up_sound = pygame.mixer.Sound("sounds\\scale-e6-14577.mp3")
except:
    # Nếu không tìm thấy file âm thanh, tạo âm thanh đơn giản
    correct_sound = pygame.mixer.Sound(pygame.sndarray.make_sound(pygame.Surface((1, 1))))
    wrong_sound = pygame.mixer.Sound(pygame.sndarray.make_sound(pygame.Surface((1, 1))))
    level_up_sound = pygame.mixer.Sound(pygame.sndarray.make_sound(pygame.Surface((1, 1))))

# Tạo gradient background với hiệu ứng sao lấp lánh
def create_gradient_background(top_color, bottom_color):
    background = pygame.Surface((WIDTH, HEIGHT))
    for y in range(HEIGHT):
        # Tính toán màu cho từng dòng pixel
        r = top_color[0] + (bottom_color[0] - top_color[0]) * y / HEIGHT
        g = top_color[1] + (bottom_color[1] - top_color[1]) * y / HEIGHT
        b = top_color[2] + (bottom_color[2] - top_color[2]) * y / HEIGHT
        pygame.draw.line(background, (r, g, b), (0, y), (WIDTH, y))
    return background

# Tạo background động với các ngôi sao lấp lánh
background = create_gradient_background((70, 130, 180), (25, 25, 112))  # SteelBlue to MidnightBlue
stars = []
for _ in range(100):
    x = random.randint(0, WIDTH)
    y = random.randint(0, HEIGHT)
    size = random.uniform(0.5, 2.0)
    brightness = random.uniform(0.3, 1.0)
    twinkle_speed = random.uniform(0.01, 0.05)
    stars.append([x, y, size, brightness, twinkle_speed])

def update_stars():
    for star in stars:
        # Làm cho độ sáng thay đổi theo sin để tạo hiệu ứng nhấp nháy
        star[3] = 0.3 + 0.7 * abs(math.sin(time.time() * star[4] * 10))
        brightness = int(255 * star[3])
        pygame.draw.circle(screen, (brightness, brightness, brightness), (int(star[0]), int(star[1])), star[2])

class Circle:
    def __init__(self, x, y, radius, number):
        self.x = x
        self.y = y
        self.radius = radius
        self.number = number
        self.show_number = True
        self.clicked = False
        self.color = BLUE
        self.shine = 0  # Hiệu ứng lấp lánh
        self.pulse_effect = 0  # Hiệu ứng nhịp đập
        self.glow_size = 0  # Kích thước hiệu ứng phát sáng
    
    def draw(self):
        # Hiệu ứng phát sáng
        if self.clicked:
            # Hiệu ứng nhịp đập cho ô đã click
            pulse = abs(math.sin(time.time() * 5)) * 5
            glow_radius = self.radius + pulse
            
            # Vẽ hiệu ứng glow
            glow_surface = pygame.Surface((glow_radius*2, glow_radius*2), pygame.SRCALPHA)
            for i in range(5):
                alpha = 100 - i * 20
                if alpha > 0:
                    pygame.draw.circle(glow_surface, (*self.color[:3], alpha), 
                                      (glow_radius, glow_radius), glow_radius - i*2)
            screen.blit(glow_surface, (self.x - glow_radius, self.y - glow_radius))
        
        # Vẽ hiệu ứng lấp lánh
        if self.shine > 0:
            shine_radius = self.radius + 5
            shine_surface = pygame.Surface((shine_radius*2, shine_radius*2), pygame.SRCALPHA)
            pygame.draw.circle(shine_surface, (*GOLD, 150), (shine_radius, shine_radius), shine_radius)
            screen.blit(shine_surface, (self.x - shine_radius, self.y - shine_radius))
            self.shine -= 1
        
        # Vẽ ô tròn với hiệu ứng 3D
        pygame.draw.circle(screen, BLACK, (self.x + 3, self.y + 3), self.radius)  # Bóng đổ
        
        # Gradient fill cho ô tròn
        if self.color == BLUE:
            for i in range(int(self.radius)):
                # Tạo gradient từ LIGHT_BLUE đến BLUE
                ratio = i / self.radius
                r = int(LIGHT_BLUE[0] * (1-ratio) + self.color[0] * ratio)
                g = int(LIGHT_BLUE[1] * (1-ratio) + self.color[1] * ratio)
                b = int(LIGHT_BLUE[2] * (1-ratio) + self.color[2] * ratio)
                pygame.draw.circle(screen, (r, g, b), (self.x, self.y), self.radius - i)
        else:
            pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)
        
        # Vẽ viền trong - tạo hiệu ứng ánh sáng
        pygame.draw.circle(screen, WHITE, (self.x - self.radius//4, self.y - self.radius//4), 
                          self.radius//4, 2)
        
        # Vẽ viền ngoài
        pygame.draw.circle(screen, WHITE, (self.x, self.y), self.radius, 2)
        
        if self.show_number:
            text = font.render(str(self.number), True, WHITE)
            text_rect = text.get_rect(center=(self.x, self.y))
            screen.blit(text, text_rect)
    
    def is_clicked(self, pos):
        distance = ((pos[0] - self.x) ** 2 + (pos[1] - self.y) ** 2) ** 0.5
        return distance <= self.radius

class Button:
    def __init__(self, x, y, width, height, text, color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover_color = (min(color[0] + 30, 255), min(color[1] + 30, 255), min(color[2] + 30, 255))
        self.active_color = color
        self.pulse = 0
    
    def draw(self):
        mouse_pos = pygame.mouse.get_pos()
        is_hover = self.rect.collidepoint(mouse_pos)
        
        # Hiệu ứng nhịp đập khi hover
        if is_hover:
            self.pulse = (self.pulse + 0.1) % (2 * math.pi)
            pulse_size = int(5 * abs(math.sin(self.pulse)))
            glow_rect = self.rect.inflate(pulse_size, pulse_size)
            
            # Vẽ hiệu ứng glow
            for i in range(3):
                border_size = 3 - i
                glow_rect_i = self.rect.inflate(pulse_size + i*4, pulse_size + i*4)
                alpha = 100 - i * 30
                s = pygame.Surface((glow_rect_i.width, glow_rect_i.height), pygame.SRCALPHA)
                pygame.draw.rect(s, (*self.hover_color, alpha), 
                                s.get_rect(), border_radius=15)
                screen.blit(s, glow_rect_i)
            
            pygame.draw.rect(screen, self.hover_color, self.rect, border_radius=10)
            pygame.draw.rect(screen, WHITE, self.rect, width=2, border_radius=10)
        else:
            # Hiệu ứng 3D khi không hover
            shadow_rect = self.rect.copy()
            shadow_rect.x += 3
            shadow_rect.y += 3
            pygame.draw.rect(screen, BLACK, shadow_rect, border_radius=10)
            pygame.draw.rect(screen, self.active_color, self.rect, border_radius=10)
            pygame.draw.rect(screen, WHITE, self.rect, width=1, border_radius=10)
        
        # Vẽ text với hiệu ứng shadow
        text_surface = font.render(self.text, True, BLACK)
        text_rect = text_surface.get_rect(center=(self.rect.centerx + 1, self.rect.centery + 1))
        screen.blit(text_surface, text_rect)
        
        text_surface = font.render(self.text, True, WHITE)
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)
    
    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

class Game:
    def __init__(self):
        self.level = 1
        self.score = 0
        self.high_score = self.load_high_score()
        self.circles = []
        self.show_numbers = True
        self.current_number = 1
        self.game_state = "menu"  # menu, ready, memorize, playing, won, lost
        self.message = "Start new game!"
        self.start_time = 0
        self.level_complete_time = 0  # Thời điểm hoàn thành level
        
        # Tạo các nút
        self.start_button = Button(WIDTH // 2 - 100, HEIGHT // 2, 200, 60, "Start", BLUE)
        self.menu_button = Button(WIDTH // 2 - 100, HEIGHT // 2 + 100, 200, 60, "Menu", PURPLE)
        self.particles = []  # Hệ thống particle
    
    def load_high_score(self):
        try:
            with open("high_score.txt", "r") as file:
                return int(file.read())
        except:
            return 0
    
    def save_high_score(self):
        try:
            with open("high_score.txt", "w") as file:
                file.write(str(self.high_score))
        except:
            pass
    
    def start_game(self):
        self.circles = []
        self.current_number = 1
        self.game_state = "memorize"
        self.message = "Remember the positions!"
        self.particles = []
        
        # Tạo n ô tròn với n = level + 2
        n = self.level + 2
        positions = []
        
        # Đảm bảo các ô tròn không chồng lên nhau
        radius = 40
        margin = 20
        max_attempts = 100
        
        for i in range(1, n + 1):
            attempts = 0
            while attempts < max_attempts:
                x = random.randint(radius + margin, WIDTH - radius - margin)
                # Đảm bảo các ô không gần với thanh thông tin ở dưới
                y = random.randint(radius + margin, HEIGHT - 120 - radius - margin)
                
                # Kiểm tra xem vị trí có chồng lên vị trí đã có không
                valid_position = True
                for pos in positions:
                    distance = ((pos[0] - x) ** 2 + (pos[1] - y) ** 2) ** 0.5
                    if distance < 2 * radius + margin:
                        valid_position = False
                        break
                
                if valid_position:
                    positions.append((x, y))
                    self.circles.append(Circle(x, y, radius, i))
                    break
                
                attempts += 1
            
            # Nếu không tìm được vị trí hợp lệ sau nhiều lần thử, đặt vị trí mặc định
            if attempts >= max_attempts:
                x = random.randint(radius + margin, WIDTH - radius - margin)
                y = random.randint(radius + margin, HEIGHT - 120 - radius - margin)
                positions.append((x, y))
                self.circles.append(Circle(x, y, radius, i))
        
        self.show_numbers = True
        self.start_time = time.time()
    
    def hide_numbers(self):
        self.show_numbers = False
        for circle in self.circles:
            circle.show_number = False
        self.game_state = "playing"
        self.message = "Click the buttons in ascending order!"
    
    def add_particles(self, x, y, color):
        # Thêm hiệu ứng particle cao cấp hơn
        for _ in range(30):
            angle = random.uniform(0, 2 * math.pi)
            speed = random.uniform(1, 5)
            size = random.uniform(2, 6)
            particle = {
                'x': x,
                'y': y,
                'dx': math.cos(angle) * speed,
                'dy': math.sin(angle) * speed,
                'radius': size,
                'color': color,
                'life': random.randint(20, 40),
                'max_life': 40,
                'alpha': 255
            }
            self.particles.append(particle)
    
    def update_particles(self):
        # Cập nhật và vẽ các particle với hiệu ứng mờ dần
        for particle in self.particles[:]:
            particle['x'] += particle['dx']
            particle['y'] += particle['dy']
            particle['radius'] *= 0.95  # Giảm kích thước theo thời gian
            particle['life'] -= 1
            
            # Giảm tốc độ theo thời gian
            particle['dx'] *= 0.95
            particle['dy'] *= 0.95
            
            if particle['life'] <= 0:
                self.particles.remove(particle)
                continue
                
            alpha = int(255 * (particle['life'] / particle['max_life']))
            color = (*particle['color'], alpha)
            
            # Vẽ particle với hiệu ứng blur
            particle_surface = pygame.Surface((int(particle['radius']*2), int(particle['radius']*2)), pygame.SRCALPHA)
            pygame.draw.circle(particle_surface, color, 
                              (int(particle['radius']), int(particle['radius'])), 
                              int(particle['radius']))
            screen.blit(particle_surface, 
                       (int(particle['x'] - particle['radius']), 
                        int(particle['y'] - particle['radius'])))
    
    def handle_click(self, pos):
        if self.game_state == "menu":
            if self.start_button.is_clicked(pos):
                self.level = 1
                self.score = 0
                self.game_state = "ready"
                self.start_game()
        elif self.game_state == "lost":
            if self.menu_button.is_clicked(pos):
                self.game_state = "menu"
        elif self.game_state == "playing":
            for circle in self.circles:
                if circle.is_clicked(pos) and not circle.clicked:
                    if circle.number == self.current_number:
                        # Đúng
                        circle.clicked = True
                        circle.color = GREEN
                        circle.show_number = True
                        circle.shine = 15  # Bắt đầu hiệu ứng lấp lánh
                        pygame.mixer.Sound.play(correct_sound)
                        self.add_particles(circle.x, circle.y, GREEN)
                        self.current_number += 1
                        self.score += 10 * self.level
                        
                        # Kiểm tra xem đã hoàn thành level chưa
                        if self.current_number > len(self.circles):
                            self.game_state = "won"
                            self.message = f"Well done! You finished level: {self.level}!"
                            self.level_complete_time = time.time()
                            pygame.mixer.Sound.play(level_up_sound)
                            # Thêm hiệu ứng mừng chiến thắng - pháo hoa
                            for _ in range(5):
                                self.add_particles(random.randint(100, WIDTH-100), 
                                                 random.randint(100, HEIGHT-200), 
                                                 (random.randint(50, 255), 
                                                  random.randint(50, 255), 
                                                  random.randint(50, 255)))
                            # Kiểm tra high score
                            if self.score > self.high_score:
                                self.high_score = self.score
                                self.save_high_score()
                    else:
                        # Sai
                        circle.color = RED
                        circle.show_number = True
                        pygame.mixer.Sound.play(wrong_sound)
                        self.add_particles(circle.x, circle.y, RED)
                        self.game_state = "lost"
                        self.message = f"Game over! Your score: {self.score}"
                        # Kiểm tra high score
                        if self.score > self.high_score:
                            self.high_score = self.score
                            self.save_high_score()
                    break
    
    def check_next_level(self):
        # Kiểm tra xem đã đến lúc chuyển level chưa
        if self.game_state == "won" and time.time() - self.level_complete_time >= 0.5:
            self.level += 1
            self.start_game()
    
    def draw_info_panel(self):
        # Vẽ hộp thông tin với viền và hiệu ứng trong suốt đẹp hơn
        info_box = pygame.Rect(10, HEIGHT - 90, 220, 80)
        
        # Vẽ nền trong suốt
        s = pygame.Surface((info_box.width, info_box.height), pygame.SRCALPHA)
        pygame.draw.rect(s, (0, 0, 50, 180), s.get_rect(), border_radius=10)
        screen.blit(s, info_box)
        
        # Vẽ viền phát sáng
        pygame.draw.rect(screen, LIGHT_BLUE, info_box, width=2, border_radius=10)
        
        # Hiển thị thông tin
        level_text = small_font.render(f"Level: {self.level}", True, WHITE)
        screen.blit(level_text, (20, HEIGHT - 80))
        
        score_text = small_font.render(f"Score: {self.score}", True, WHITE)
        screen.blit(score_text, (20, HEIGHT - 60))
        
        high_score_text = small_font.render(f"Record: {self.high_score}", True, WHITE)
        screen.blit(high_score_text, (20, HEIGHT - 40))

def main():
    clock = pygame.time.Clock()
    game = Game()
    
    running = True
    while running:
        # Vẽ background với hiệu ứng sao lấp lánh
        screen.blit(background, (0, 0))
        update_stars()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                game.handle_click(event.pos)
        
        # Ẩn số sau thời gian (tùy theo level, càng cao càng ít thời gian)
        memorize_time = max(0.8, 2.0 - game.level * 0.1)  # Giảm dần theo level, tối thiểu 0.8s
        if game.game_state == "memorize" and time.time() - game.start_time > memorize_time:
            game.hide_numbers()
        
        # Kiểm tra chuyển level sau hoàn thành
        if game.game_state == "won":
            game.check_next_level()
        
        # Vẽ các ô tròn
        for circle in game.circles:
            circle.draw()
        
        # Cập nhật và vẽ particles
        game.update_particles()
        
        # Hiển thị thông tin
        if game.game_state != "menu":
            game.draw_info_panel()
        
        # Hiển thị thông báo - di chuyển lên trên để không che lấp thông tin
        if game.message:
            message_bg = pygame.Rect(WIDTH // 2 - 300, 10, 600, 50)
            
            # Vẽ nền trong suốt
            s = pygame.Surface((message_bg.width, message_bg.height), pygame.SRCALPHA)
            pygame.draw.rect(s, (0, 0, 50, 200), s.get_rect(), border_radius=10)
            screen.blit(s, message_bg)
            
            # Viền sáng
            pygame.draw.rect(screen, LIGHT_BLUE, message_bg, width=2, border_radius=10)
            
            message_text = font.render(game.message, True, WHITE)
            message_rect = message_text.get_rect(center=(WIDTH // 2, 10 + message_bg.height // 2))
            screen.blit(message_text, message_rect)
        
        # Hiển thị các nút và giao diện khác
        if game.game_state == "menu":
            # Vẽ tiêu đề với hiệu ứng sáng
            title_text = "Memory Game"
            # Vẽ glow đằng sau text
            for i in range(10, 0, -2):
                alpha = 25 * (1 - i/10)
                glow_text = title_font.render(title_text, True, (*GOLD, alpha))
                glow_rect = glow_text.get_rect(center=(WIDTH // 2, HEIGHT // 4))
                glow_rect.inflate_ip(i*2, i*2)
                screen.blit(glow_text, glow_rect)
            
            # Vẽ bóng đổ
            title_shadow = title_font.render(title_text, True, BLACK)
            shadow_rect = title_shadow.get_rect(center=(WIDTH // 2 + 4, HEIGHT // 4 + 4))
            screen.blit(title_shadow, shadow_rect)
            
            # Vẽ text chính
            title_text_surface = title_font.render(title_text, True, GOLD)
            title_rect = title_text_surface.get_rect(center=(WIDTH // 2, HEIGHT // 4))
            screen.blit(title_text_surface, title_rect)
            
            # Vẽ nút bắt đầu
            game.start_button.draw()
            
            # Hiển thị hướng dẫn
            instructions = [
                "Remember the positions of numbers",
                "Click them in ascending order (1, 2, 3...)",
                "Complete levels to score points!"
            ]
            
            for i, instruction in enumerate(instructions):
                instr_text = small_font.render(instruction, True, WHITE)
                instr_rect = instr_text.get_rect(center=(WIDTH // 2, HEIGHT // 2 + 100 + i*25))
                screen.blit(instr_text, instr_rect)
        
        elif game.game_state == "lost":
            game.menu_button.draw()
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

SystemExit: 